In [1]:
import torch
from torch.utils.data import Dataset
import scanpy as sc
import scvelo as scv
import pandas as pd
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch import nn

import model

#### DLPFC

In [4]:
scrna_path = "DLPFC_151673\\scrna.h5ad"
st_path = "DLPFC_151673\\filtered_feature_bc_matrix.h5"
st_full_path = "DLPFC_151673"

In [5]:
scrna = scv.read(scrna_path)
st = sc.read_visium(st_full_path)

In [6]:
scrna.to_df()

,RP11-34P13.3,RP11-34P13.7,RP11-34P13.14,FO538757.3,FO538757.2,AP006222.2,RP5-857K21.15,RP4-669L17.2,RP4-669L17.10,RP5-857K21.4,...,AC011043.2,AL592183.1,AC007325.4,AC007325.2,BX072566.1,AL354822.1,AC023491.2,AC004556.1,AC240274.1,FAM231B
Ex_10_L2_4.3_Control_B3_AAACCTGAGGTAGCCA,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Inhib_5.3_Control_B3_AAACCTGCAAACTGTC,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex_10_L2_4.3_Control_B3_AAACCTGCAACAACCT,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex_8_L5_6.3_Control_B3_AAACCTGGTCCGAACC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Inhib_2_VIP.3_Control_B3_AAACCTGGTCGTGGCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OPCs_1.17_Suicide_B1_TTTGTCAAGCTCCTTC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex_10_L2_4.17_Suicide_B1_TTTGTCAAGGCTAGCA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ex_10_L2_4.17_Suicide_B1_TTTGTCAAGGGATGGG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Inhib_2_VIP.17_Suicide_B1_TTTGTCACATGTCCTC,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
st.to_df()

,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.4,AL732372.1,OR4F29,AC114498.1,...,AC007325.2,BX072566.1,AL354822.1,AC023491.2,AC004556.1,AC233755.2,AC233755.1,AC240274.1,AC213203.1,FAM231C
AAACAAGTATCTCCCA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAATCTACTAGCA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACACCAATAACTGC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGAGCGACTCCT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCTTTCAGAAG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTGTTTCATTAGTCTA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTGTTTCCATACAACT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTGTTTGTATTACACG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
st.obs

,in_tissue,array_row,array_col
AAACAAGTATCTCCCA-1,1,50,102
AAACAATCTACTAGCA-1,1,3,43
AAACACCAATAACTGC-1,1,59,19
AAACAGAGCGACTCCT-1,1,14,94
AAACAGCTTTCAGAAG-1,1,43,9
...,...,...,...
TTGTTTCACATCCAGG-1,1,58,42
TTGTTTCATTAGTCTA-1,1,60,30
TTGTTTCCATACAACT-1,1,45,27
TTGTTTGTATTACACG-1,1,73,41


In [13]:
data = model.SingleCellDataset(scrna_path)

,array_row,array_col
AAACAAGTATCTCCCA-1,50,102
AAACAATCTACTAGCA-1,3,43
AAACACCAATAACTGC-1,59,19
AAACAGAGCGACTCCT-1,14,94
AAACAGCTTTCAGAAG-1,43,9
...,...,...
TTGTTTCACATCCAGG-1,58,42
TTGTTTCATTAGTCTA-1,60,30
TTGTTTCCATACAACT-1,45,27
TTGTTTGTATTACACG-1,73,41


#### Chicken heart

In [3]:
scrna_path = "chicken_heart\\RNA_D14_adata.h5ad"
st_path = "chicken_heart\\Visium_D14_adata.h5ad"

In [4]:
scrna = scv.read(scrna_path)
st = scv.read(st_path)

In [11]:
st.to_df("spliced")

,C10orf88,ARID4A,CTRB2,TUBB3,SPR,ENSGALG00000000071,TIMM17A,INTS4,ADIPOR1,UBE2T,...,ENSGALG00000051933,ENSGALG00000050868,ENSGALG00000049962,TCEA3,ENSGALG00000053489,ENSGALG00000050908,ENSGALG00000047547,ENSGALG00000054746,CCDC113,ENSGALG00000049917
AAACAAGTATCTCCCA,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.122402,0.0,0.000000,0.707467,...,0.0,0.000000,0.0,1.414935,0.0,0.0,0.0,0.000000,0.0,0.0
AAACATTTCCCGGATT,0.000000,0.000000,0.0,0.000000,0.000000,0.0,3.296033,0.0,0.659207,0.000000,...,0.0,1.977620,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
AAACCCGAACGAAATC,1.420402,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
AAACCGTTCGTCCAGG,0.000000,0.880268,0.0,0.880268,0.000000,0.0,3.521073,0.0,0.880268,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
AAACCTAAGCAGCCGG,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.961901,0.0,0.000000,0.000000,...,0.0,0.961901,0.0,0.000000,0.0,0.0,0.0,0.961901,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTTTCCCGAAAG,0.000000,0.638771,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.638771,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
TTGTTAGCAAATTCGA,0.000000,1.121142,0.0,0.000000,0.000000,0.0,2.242284,0.0,1.121142,1.121142,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
TTGTTCAGTGTGCTAC,0.793473,0.000000,0.0,0.000000,0.000000,0.0,1.586945,0.0,0.793473,1.586945,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
TTGTTGTGTGTCAAGA,2.422246,0.000000,0.0,0.000000,1.211123,0.0,1.211123,0.0,0.000000,1.211123,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [10]:
st.to_df("unspliced")

,C10orf88,ARID4A,CTRB2,TUBB3,SPR,ENSGALG00000000071,TIMM17A,INTS4,ADIPOR1,UBE2T,...,ENSGALG00000051933,ENSGALG00000050868,ENSGALG00000049962,TCEA3,ENSGALG00000053489,ENSGALG00000050908,ENSGALG00000047547,ENSGALG00000054746,CCDC113,ENSGALG00000049917
AAACAAGTATCTCCCA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACATTTCCCGGATT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCCGAACGAAATC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCGTTCGTCCAGG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTAAGCAGCCGG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTTTCCCGAAAG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTGTTAGCAAATTCGA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.254697,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTGTTCAGTGTGCTAC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTGTTGTGTGTCAAGA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
st.obsm['X_xy_loc']

array([[16790.5, 13596.5],
       [16103.5, 16250.5],
       [18589.5, 12387.5],
       ...,
       [11512.5,  7338.5],
       [13317.5,  9022.5],
       [ 8480.5, 15542.5]])

In [12]:
st

AnnData object with n_obs × n_vars = 1967 × 12295
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'percent.mito', 'Spatial_snn_res.1', 'seurat_clusters', 'scanorama_snn_res.1', 'Cardiomyocytes-1', 'Cardiomyocytes-2', 'Immature.myocardial.cells', 'Valve.cells', 'Macrophages', 'Fibroblast.cells', 'Erythrocytes', 'Endocardial.cells', 'MT-enriched.cardiomyocytes', 'Epi-epithelial.cells', 'Vascular.endothelial.cells', 'TMSB4X.high.cells', 'Epi-mesenchymal.cells', 'Dendritic.cells', 'Mural.cells', 'max', 'celltype_prediction_max', 'celltype_prediction', 'celltype_prediction_mode', 'region', 'n_counts'
    var: 'gene_ids', 'n_cells', 'gene_count_corr'
    uns: 'celltype_prediction_colors'
    obsm: 'X_xy_loc'
    layers: 'spliced', 'unspliced'